In [ ]:
!pip install -flask pyngrok flask_ngrok line-bot-sdk

In [ ]:
!pip install -flask pyngrok flask_ngrok line-bot-sdk

# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_auth'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage
)

app = Flask(__name__)
configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content("question")
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
                #reply sticker
             reply=StickerMessage(package_id='446',sticker_id='1989')
        elif action =='image':
                #reply image
                reply =ImageMessage(
                    original_content_url='https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280-',
                    previw_image_url=''https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg'
                    )
                pass
        elif action == 'video':
          reply = VideoMessage (
            original_content_url= 'https://cdn.pixabay.com/video/2021/07/08/80724-573496156_large.mp4',
            preview_image_url='https://cdn-pixabay.com/photo/2017/02/11/14/45/taipei-taiwan-2057818_1280-jpg')

        elif action == 'location':
          reply = LocationMessage (
          title='Taipei 101',
          address='110台北市信義區信義路五段7號'，
          latitude=25.034199559170606,
          longitude=121.56456035396677
          )
        else:
            response = ask_gemini(action)
            reply = TextMessage(text=response)


        line_bot_api.reply_message(
            #reply area
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() # for colab
if __name__ == "__main__":
    app.run()